
Dataset competition `https://www.kaggle.com/competitions/ga-customer-revenue-prediction/`

Columns description `https://support.google.com/analytics/answer/3437719?hl=en`


In [1]:
import pandas as pd
from pathlib import Path
import json
from pandas import json_normalize
import numpy as np

# Load and prepare the data

adopted (with changes) from `https://www.kaggle.com/code/ashrafkhan94/googleanalytics-customer-revenue-eda-prediction/notebook`


### 1. Reading datafile (handling columns with Json Values)

In [2]:
columns = ['device', 'geoNetwork', 'totals', 'trafficSource'] # Columns that have json format

dir_path = Path('/anvar/public_datasets/ga-customer-revenue-data')

# p is a fractional number to skiprows and read just a random sample of the our dataset. 
p = 0.2 # *** In this case we will use 20% of data set *** #

#Code to transform the json format columns in table
def json_read(filename, random_state=17):
    data_frame = dir_path / filename
    random = np.random.RandomState(random_state)
    
    #Importing the dataset
    df = pd.read_csv(data_frame, 
                     converters={column: json.loads for column in columns}, # loading the json columns properly
                     dtype={'fullVisitorId': 'str'}, # transforming this column to string
                     skiprows=lambda i: i>0 and random.random() > p)
    
    for column in columns: #loop to finally transform the columns in data frame
        #It will normalize and set the json to a table
        column_as_df = json_normalize(df[column]) 
        # here will be set the name using the category and subcategory of json columns
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns] 
        # after extracting the values, let drop the original columns
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    return df 

In [3]:
df = json_read('train_v2.csv')

### 2. Function to Describe each column

In [4]:
def DataDesc(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values
    return summary

In [5]:
DataDesc(df)

Dataset Shape: (341831, 59)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value
0,channelGrouping,object,0,8,Direct,Organic Search,Organic Search
1,customDimensions,object,0,6,"[{'index': '4', 'value': 'North America'}]","[{'index': '4', 'value': 'EMEA'}]","[{'index': '4', 'value': 'Central America'}]"
2,date,int64,0,638,20171016,20171016,20171016
3,fullVisitorId,object,0,311212,7992466427990357681,9075655783635761930,632878546807742341
4,hits,object,0,298377,"[{'hitNumber': '1', 'time': '0', 'hour': '17',...","[{'hitNumber': '1', 'time': '0', 'hour': '9', ...","[{'hitNumber': '1', 'time': '0', 'hour': '19',..."
5,socialEngagementType,object,0,1,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged
6,visitId,int64,0,339614,1508201613,1508169851,1508207516
7,visitNumber,int64,0,314,1,1,1
8,visitStartTime,int64,0,339668,1508201613,1508169851,1508207516
9,device.browser,object,0,54,Chrome,Chrome,Chrome




- Notice there are a lot of constant columns having only one unique value
- Target column = totals.transactionRevenue (Null value = No Revenue)
- Columns with prefix trafficSource contain Null values majorly, hence we will drop these columns



### 3. Describing Columns with Null Values

In [6]:
def Null_Count(df):
    df_null = df.isnull().sum().sort_values(ascending = False).rename('Null').reset_index()

    null_count = df_null['Null']
    null_percent = (null_count * 100) / (df.shape[0])

    df_null = pd.concat([df_null['index'], null_count,null_percent], axis=1,
                        keys=['Column','Null_Count','Null_Percent'])

    return df_null[df_null['Null_Count'] != 0]

Null_Count(df)

,Column,Null_Count,Null_Percent
0,totals.totalTransactionRevenue,338138,98.919642
1,totals.transactionRevenue,338138,98.919642
2,totals.transactions,338125,98.915839
3,trafficSource.adContent,328759,96.175888
4,trafficSource.adwordsClickInfo.isVideoAd,326641,95.556284
5,trafficSource.adwordsClickInfo.adNetworkType,326641,95.556284
6,trafficSource.adwordsClickInfo.slot,326641,95.556284
7,trafficSource.adwordsClickInfo.page,326641,95.556284
8,trafficSource.adwordsClickInfo.gclId,326609,95.546922
9,trafficSource.isTrueDirect,234916,68.722848


### 4. Function to Fill Null Values

In [7]:
def fill_na(df):   
    df['totals.pageviews'].fillna(1, inplace=True)
    df['totals.newVisits'].fillna(0, inplace=True)
    df['totals.bounces'].fillna(0, inplace=True) 
    df['totals.transactionRevenue'].fillna(0.0, inplace=True)
    df['totals.totalTransactionRevenue'].fillna(0.0, inplace=True)
    df['totals.transactions'].fillna(0, inplace=True)
    df['totals.timeOnSite'].fillna(0, inplace=True)
    df['totals.sessionQualityDim'].fillna(0, inplace=True)
    
    # Changing datatypes from object to desired ones
    df['totals.pageviews'] = df['totals.pageviews'].astype(np.int32)
    df['totals.newVisits'] = df['totals.newVisits'].astype(np.int32)
    df['totals.bounces'] = df['totals.bounces'].astype(np.int32)
    df['totals.transactionRevenue'] = df['totals.transactionRevenue'].astype(np.float32)
    df['totals.totalTransactionRevenue'] = df['totals.totalTransactionRevenue'].astype(np.float32)
    df['totals.transactions'] = df['totals.transactions'].astype(np.int32)
    df['totals.sessionQualityDim'] = df['totals.sessionQualityDim'].astype(np.float32)
    df['totals.hits'] = df['totals.hits'].astype(np.int32)
    df['totals.timeOnSite'] = df['totals.timeOnSite'].astype(np.float32)
    
    df['trafficSource.isTrueDirect'].fillna(False, inplace=True) 
    df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True
    df[df['geoNetwork.city'] == "(not set)"]['geoNetwork.city'] = np.nan
    df['geoNetwork.city'].fillna("NaN", inplace=True)

    df['visitNumber'] = df['visitNumber'].astype(np.int32)

  
    return df

df = fill_na(df)

<ipython-input-7-e9afd5968a6e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['geoNetwork.city'] == "(not set)"]['geoNetwork.city'] = np.nan


### 5. Processing Date/Time Columns

In [8]:
from datetime import datetime

# This function is to extract date features
def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # seting the column as pandas datetime
    df["weekday"] = df['date'].dt.weekday #extracting week day
    df["day"] = df['date'].dt.day # extracting day
    df["month"] = df['date'].dt.month # extracting day
    df["year"] = df['date'].dt.year # extracting day
    df['visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df.drop('date', axis=1)

df = date_process(df)

### 6. Removing constant columns

In [9]:
constant_columns = [col for col in df.columns if df[col].nunique() == 1]

for col in constant_columns:
    print(col)
print(f'Num of Columns : {len(constant_columns)}')

df.drop(constant_columns, axis=1, inplace=True)

socialEngagementType
device.browserVersion
device.browserSize
device.operatingSystemVersion
device.mobileDeviceBranding
device.mobileDeviceModel
device.mobileInputSelector
device.mobileDeviceInfo
device.mobileDeviceMarketingName
device.flashVersion
device.language
device.screenColors
device.screenResolution
geoNetwork.cityId
geoNetwork.latitude
geoNetwork.longitude
geoNetwork.networkLocation
totals.visits
trafficSource.adwordsClickInfo.criteriaParameters
Num of Columns : 19


### 7. Dropping Null Columns

In [10]:
nulls = Null_Count(df)
nulls

,Column,Null_Count,Null_Percent
0,trafficSource.adContent,328759,96.175888
1,trafficSource.adwordsClickInfo.page,326641,95.556284
2,trafficSource.adwordsClickInfo.adNetworkType,326641,95.556284
3,trafficSource.adwordsClickInfo.slot,326641,95.556284
4,trafficSource.adwordsClickInfo.gclId,326609,95.546922
5,trafficSource.referralPath,228671,66.895922
6,trafficSource.keyword,210664,61.628114


In [11]:
cols_to_drop = ['trafficSource.adContent', 'trafficSource.adwordsClickInfo.page',
                'trafficSource.adwordsClickInfo.adNetworkType',
                'trafficSource.adwordsClickInfo.slot',
                'trafficSource.adwordsClickInfo.gclId',
                'trafficSource.referralPath', 'trafficSource.keyword',
                'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.metro',
               'visitStartTime', 'visitId']

df.drop(cols_to_drop, axis=1, inplace=True)

### 8. Serialize remaining json columns

In [12]:
def split_json(x):
    dictionary = json.loads(x.replace("'", '"'))
    if len(dictionary) > 0:
        return dictionary[0]
    else:
        return {'index': np.nan, 'value':'Unknown'}

custom_dimensions = df.customDimensions.apply(split_json)
custom_dimensions_cols = pd.DataFrame.from_records(custom_dimensions.values).rename(
    columns={'index': 'customDimensions.index', 'value':'customDimensions.value'})

df = df.merge(custom_dimensions_cols, right_index=True, left_index=True)
df.drop(['hits', 'customDimensions', 'customDimensions.index', 'totals.transactions'], axis=1, inplace=True)

### 9. Prepare target column

In [13]:
df.drop(['totals.transactionRevenue'], axis=1, inplace=True)

### 10. Prep bool columns


In [14]:
df['device.isMobile'] = df['device.isMobile'].astype(np.int32)
df['trafficSource.isTrueDirect'] = df['trafficSource.isTrueDirect'].astype(np.int32)
df['trafficSource.adwordsClickInfo.isVideoAd'] = df['trafficSource.adwordsClickInfo.isVideoAd'].astype(np.int32)

#  Split and save data for HA

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
train_id, test_id = train_test_split(df.fullVisitorId, train_size=0.7)
train, test = df[df.fullVisitorId.isin(train_id)], df[df.fullVisitorId.isin(test_id)]

In [17]:
def return_X_y(df):
    X = df.drop(['fullVisitorId', 'totals.totalTransactionRevenue'], axis=1)    
    y = df['totals.totalTransactionRevenue']
    return X, y

X_train, y_train = return_X_y(train)
X_test, y_test = return_X_y(test)

In [18]:
X_train.merge(y_train, right_index=True, left_index=True).reset_index(drop=True).to_csv('/home/anvar/ha2_train.csv', index=False)
X_test.merge(y_test, right_index=True, left_index=True).reset_index(drop=True).to_csv('/home/anvar/ha2_test.csv', index=False)

In [19]:
df_train = pd.read_csv('/home/anvar/ha2_train.csv')
df_train.head()

,channelGrouping,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.city,...,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,weekday,day,month,year,visitHour,customDimensions.value,totals.totalTransactionRevenue
0,Organic Search,1,Chrome,Macintosh,0,desktop,Americas,Central America,Mexico,Mexico City,...,organic,0,1,0,16,10,2017,5,Central America,0.0
1,Organic Search,1,Chrome,Windows,0,desktop,Americas,Northern America,Canada,not available in demo dataset,...,organic,0,1,0,16,10,2017,20,North America,0.0
2,Organic Search,1,Chrome,Linux,0,desktop,Asia,Eastern Asia,Taiwan,not available in demo dataset,...,organic,0,1,0,16,10,2017,9,APAC,0.0
3,Organic Search,1,Chrome,Android,1,mobile,Americas,Northern America,Canada,not available in demo dataset,...,organic,0,1,0,16,10,2017,4,North America,0.0
4,Organic Search,1,Chrome,Linux,0,desktop,Asia,Southern Asia,India,not available in demo dataset,...,organic,0,1,0,16,10,2017,22,APAC,0.0


In [20]:
df_train.shape

(251396, 28)

In [21]:
df_test = pd.read_csv('/home/anvar/ha2_test.csv')
df_test.head()

,channelGrouping,visitNumber,device.browser,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.city,...,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,weekday,day,month,year,visitHour,customDimensions.value,totals.totalTransactionRevenue
0,Direct,1,Chrome,Android,1,mobile,Americas,Northern America,United States,not available in demo dataset,...,(none),1,1,0,16,10,2017,3,North America,0.0
1,Organic Search,1,Chrome,Windows,0,desktop,Asia,Western Asia,Turkey,not available in demo dataset,...,organic,0,1,0,16,10,2017,19,EMEA,0.0
2,Organic Search,1,Safari,Macintosh,0,desktop,Europe,Northern Europe,Sweden,not available in demo dataset,...,organic,0,1,0,16,10,2017,16,EMEA,0.0
3,Organic Search,1,Safari,iOS,1,mobile,Americas,Northern America,United States,not available in demo dataset,...,organic,0,1,0,16,10,2017,1,North America,0.0
4,Organic Search,1,Safari,Macintosh,0,desktop,Europe,Western Europe,Belgium,not available in demo dataset,...,organic,0,1,0,16,10,2017,11,EMEA,0.0


In [22]:
df_test.shape

(118506, 28)